# Contents
This notebook carries out the following steps sequentially:
- load data from cellranger (adata.X, GEX-matrix), the SLAM count matrices (new/old, i.e. labeled/unlabeled), and un-/spliced count matrices generated by velocyto
- demultiplexing (from HTOdemux in Seurat) / annotation of sample identity (donor) and condition (perturbation). Throws out doublets and negatives from demuxing.
- preprocess the data:
    - (Filtering was done in previous steps / before demultiplexing)
    - Normalization of X and the layers, log-normalization
    - scoring of gene sigantures / cell cycle
    - cell cycle regression (depending on user settings. We produced datasets both with and without cc regression).
- preparation for visualization
    - pca
    - KNN
    - umap (based on top 2000 HVGs)
    - diffmap (based on top 2000 HVGs)
- steady state models of both SLAM and RNA velocity + velocity graph (embeddings)
- adding of progeny scores from external tool (PROGENY in R)
- division into seperate donor datasets, then saving them as .h5

# Imports and Settings (run these first!)

In [1]:
import scvelo as scv  # 0.1.16.dev32
from IPython.display import clear_output
import matplotlib.backends.backend_pdf
from tqdm import tnrange, tqdm_notebook
import scanpy as sc
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline


scv.logging.print_version()
scv.settings.set_figure_params(
    'scvelo', dpi_save=100, dpi=80, transparent=True)
scv.settings.verbosity = 2

Running scvelo 0.1.16.dev32+c00a55e.dirty (python 3.6.6) on 2021-01-28 09:48.


In [2]:
# Please adapt your paths accordingly (Warning: requires ~50GB space)
data_path='G:/data/scSLAMseq/revision/'
signatures_path='G:/data/scrnaseq_signature_collection/'
libraries = ['AB', 'CE', 'DF']
donors=['B2-040', 'C2-019', 'OT227', 'OT302', 'P009T', 'P013T']

# Add layers, annotate identities, split into donors

Input:
- libraries: `data_path+'cellranger_output_MM_ML_revision_'+library+'/outs/filtered_feature_bc_matrix.h5'`
- SLAM layers: `data_path+'slam_MM_ML_'+library+'/SLAM_PIPELINE/'+time+'_matrix'`
- velocyto looms: `data_path+'looms/'+library+'_loom/'+library+'.loom'`
- annotation file from demuxing and filtering (seurat): `data_path+'annotation.tsv'`

Output: `data_path+'by_donors/SLAMv2_'+donor+'.h5'`

In [51]:
# Define functions:

# This function loads SLAM layers and libraries, aligns gene and cell names (alphabetically) and merges them together
def load_and_sparse_add_SLAM(library='CE'):
    from scipy.sparse import csr_matrix
    adata = sc.read_10x_h5(data_path+'cellranger_output_MM_ML_revision_'+library+'/outs/filtered_feature_bc_matrix.h5')
    adata.var_names_make_unique()
    for time in ['new', 'old']:
        bdata=sc.read_10x_mtx(data_path+'slam_MM_ML_'+library+'/SLAM_PIPELINE/'+time+'_matrix')
        bdata.var_names_make_unique()
        
        d = {j: np.where(adata.var_names==gene)[0][0] for j, gene in enumerate(tqdm_notebook(bdata.var_names))}  # maps bdata vars to adata vars indices

        # map indices
        new_indices = []
        for i in tqdm_notebook(range(len(bdata.X.indptr)-1)):
            new_indices.extend(list(map(d.get, bdata.X.indices[bdata.X.indptr[i]:bdata.X.indptr[i+1]])))

        # cells are already aligned it seems
        assert np.sum(bdata.obs_names != adata.obs_names)==0

        M=csr_matrix((bdata.X.data, new_indices, bdata.X.indptr), adata.X.shape)
        adata.layers[time]=M
    return adata

# Adds the u & s layers from velocyto for "classical" RNA velocity
def add_real_loom(library, adata=None):
    adata = sc.read_10x_h5(data_path+'cellranger_output_MM_ML_revision_'+library+'/outs/filtered_feature_bc_matrix.h5') if adata is None else adata
    bdata = scv.read_loom(data_path+'looms/'+library+'_loom/'+library+'.loom')
    
    # clean names
    adata.obs_names = [x.split('-')[0] for x in adata.obs_names]
    bdata.obs_names = [x.split(':')[-1] for x in bdata.obs_names]
    adata.var_names_make_unique()
    bdata.var_names_make_unique()

    # subset to same genes and cells
    bdata = bdata[np.isin(bdata.obs_names, adata.obs_names)].copy()
    bdata = bdata[:, np.isin(bdata.var_names, adata.var_names)].copy()
    adata = adata[np.isin(adata.obs_names, bdata.obs_names)].copy()
    adata = adata[:, np.isin(adata.var_names, bdata.var_names)].copy()
    
    assert adata.n_obs==bdata.n_obs
    assert adata.n_vars==bdata.n_vars
    
    # align genes and cells (alphabetically sorted)
    av=np.argsort(adata.var_names)
    ao=np.argsort(adata.obs_names)
    adata = adata[ao,av].copy()
    bv=np.argsort(bdata.var_names)
    bo=np.argsort(bdata.obs_names)
    bdata = bdata[bo,bv].copy()
    
    # add layers from loom
    adata.layers['real_unspliced']=bdata.layers['unspliced']
    adata.layers['real_spliced']=bdata.layers['spliced']
    adata.layers['real_ambiguous']=bdata.layers['ambiguous']
    return adata

# Applies filtering and demultiplexing (done in Seurat)
def annotate_identity(library, adata = None):
    # load
    adata = sc.read_10x_h5(data_path+'cellranger_output_MM_ML_revision_'+library+'/outs/filtered_feature_bc_matrix.h5') if adata is None else adata
    adata.var_names_make_unique()
    adata.obs_names = [x.split('-')[0] for x in adata.obs_names]

    # add annotation
    tab = pd.read_csv(data_path+'annotation.tsv', sep='\t')
    subtab=tab[tab.cell.str.startswith(library)]
    subtab.cell=subtab.cell.str.replace(library+'_', '')
    subtab['library'] = library
    subtab = subtab.set_index('cell')
    subtab = subtab[np.isin(subtab.index, adata.obs_names)]  # subset obs
    adata.obs=pd.concat([adata.obs, subtab], axis=1, join='outer')

    # throw out cell that nils filtered and hence did not annotate
    adata = adata[~pd.isna(adata.obs.library)]

    # throw out Doublets and negatives from HTO demux
    adata = adata[adata.obs['HTO_classification.global']=='Singlet'].copy()
    
    # annotate ribosomal
    ribo_genes = np.logical_or(adata.var_names.str.startswith('RPS'), adata.var_names.str.startswith('RPL'))
    adata.obs['percent_ribosomal'] = np.sum(adata[:, ribo_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
    
    return adata
        

In [ ]:
from pathlib import Path
Path(data_path+'by_donors/').mkdir(parents=True, exist_ok=True)
# Add SLAM and velocyto layers, apply demux and filtering from Seurat, split libraries into donors
for library in tqdm_notebook(libraries):
    adata = load_and_sparse_add_SLAM(library)
    print(vars(adata.X))  # this is for control
    adata = add_real_loom(library, adata)
    print(vars(adata.X))
    adata = annotate_identity(library, adata)
    print(vars(adata.X))
    for donor in pd.unique(adata.obs.organoid):
        adata[adata.obs.organoid==donor].write(data_path+'by_donors/SLAMv2_'+donor+'.h5')
        print(vars(adata.X))

# Signature Scoring, Preprocessing & SS velocities

input: 
- `data_path+'by_donors/SLAMv2_'+donor+'.h5'` from previous step.
- Signatures (found in the respective references)

output: `data_path+'/by_donors/processed/SLAMv2_'+donor+'_processed(_ccreg).h5'`

In [4]:
# Define functions:

# normalize and log transform GEX matrix and layers
def prepare(adata):
    sc.pp.normalize_total(adata)  # normalize X
    scv.pp.normalize_per_cell(adata, layers=['old', 'new', 'real_unspliced', 'real_spliced'])
    scv.pp.log1p(adata)  # enforce normalize X
    return adata

# Annotate data with Cell cycle score (+ cc regression if selected), ISC signatures, 
# Custom Signatures (Florian Uhlitz, YAP), Hallmark
def annotate(adata, regress_cc=False):
    # Annotations
    # single genes of interest from Markus Morkel
    single_genes = ['LGR5', 'OLFM4', 'TFF3', 'FABP1', 'EPHB2', 'AXIN1', 'AXIN2', 'EGR1']
    
    # Locally set silent, cuz it's annoying :)
    k = sc.settings.verbosity
    sc.settings.verbosity = 0

    # cc score, get it from https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt
    cell_cycle_genes = [x.strip() for x in open(signatures_path+'cell_cycle_genes/regev_lab_cell_cycle_genes.txt')]
    s_genes = cell_cycle_genes[:43]
    g2m_genes = cell_cycle_genes[43:]
    cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
    adata.obs_names_make_unique()
    sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)
    if regress_cc:
        sc.pp.regress_out(adata, ['S_score', 'G2M_score'])

    # Stem sig from curated lists, see references
    tab=pd.read_excel(signatures_path+'cell_type_markers/CRC-related_stem_cell_signatures.xlsx', header=0)
    tab = tab.drop(0)
    sigs = {'Stem_'+x: list(tab[x][~pd.isna(tab[x])].values) for x in tab.columns}
    for ct in ['Stem_Lgr5_ISC-Munoz', 'Stem_Lgr5_ISC-Merlos']:
        sc.tl.score_genes(adata, sigs[ct], score_name=ct)

    # Flo sig, from this publication
    tab=pd.read_excel(signatures_path+'cell_type_markers/colonoid_cancer_uhlitz_markers_revised.xlsx', header=1, sheet_name=2)
    flo_sigs={x: list(tab[tab['cell_type_epi']==x].gene.values) for x in pd.unique(tab['cell_type_epi'])}
    for ct in ['Stem', 'Enterocytes 1', 'Enterocytes 2', 'TC1', 'TC2', 'TC3', 'TC4', 'Goblet', 'Stem/TA 1', 'Stem/TA 2', 'Stem/TA 3']:  #flo_sigs.keys():
        sc.tl.score_genes(adata, flo_sigs[ct], score_name=ct)
    
    # Hallmarks, http://www.gsea-msigdb.org/gsea/msigdb/collections.jsp
    tab=pd.read_csv(signatures_path + 'msigdb/hallmark/h.all.v6.2.symbols.gmt', sep='\t', index_col=0, header=None).drop(1, axis=1).T
    hallsigs={hallmark : tab[hallmark][~pd.isna(tab[hallmark])].values for hallmark in tab.columns}
    for hm in ['HALLMARK_DNA_REPAIR', 'HALLMARK_WNT_BETA_CATENIN_SIGNALING']:
        sc.tl.score_genes(adata, hallsigs[hm], score_name=hm)
        
    # YAP targets (custom, curated)
    yap_targets = ['CTGF', 'GGTA1', 'WWC2', 'ANXA8', 'CLU', 'CXCL16', 'IL33', 'LY6A', 'LY6C1', 'MSLN', 'TNFRSF12A', 'CTGF', 'GGTA1', 'WWC2', 'ANXA5', 'TACSTD2', 'ANXA10', 'EREG', 'IL33', 'ANXA1', 'ANXA3']
    sc.tl.score_genes(adata, yap_targets, score_name='YAP_targets')

    sc.settings.verbosity = k
    return adata

# PCA, KNN, and UMAP/diffmap based on top 2000 HVGs
def embedd(adata):
    scv.pp.pca(adata)
    scv.pp.neighbors(adata)

    # umap on 2000 HVGs
    bdata=scv.pp.filter_genes_dispersion(adata, n_top_genes=2000, copy=True)
    scv.pp.pca(bdata)
    scv.pp.neighbors(bdata)
    scv.tl.umap(bdata)
    scv.tl.diffmap(bdata)
    adata.obsm['X_umap']=bdata.obsm['X_umap']
    adata.obsm['X_diffmap']=bdata.obsm['X_diffmap']
    del bdata
    return adata

# Make the KNN disjoint w.r.t. perturbations. Since the different perturbations came from different replicates, 
# we would mask out variation when applying the moment estimation on a complete KNN graph.
# See methods for details.
def restrict_KNN(adata):
    # restrict KNN connectivities to within perturbations only
    from scipy.sparse import csr_matrix
    adata = adata[np.argsort(adata.obs.perturbation)].copy()  # sort by perturbation
    A=adata.obsp['connectivities'].A
    for pert in pd.unique(adata.obs.perturbation):
        # identify nodes
        a=np.where(adata.obs.perturbation==pert)[0]
        b=np.where(adata.obs.perturbation!=pert)[0]
        # remove edges to different perturbations
        A[np.min(a):np.max(a), b]=0
        A[b, np.min(a):np.max(a)]=0
    adata.obsp['connectivities'] = csr_matrix(A)
    return adata

# Compute steady state velocities, velocity graphs and velocity embeddings.
def velocity(adata):
    adata = restrict_KNN(adata)
    
    # SLAM
    adata.layers['unspliced']=adata.layers['new']
    adata.layers['spliced']=adata.layers['old']
    scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    scv.tl.velocity(adata, vkey='SLAM_velocity')
    scv.tl.velocity_graph(adata, vkey='SLAM_velocity')
    scv.tl.velocity_embedding(adata, basis='umap', vkey='SLAM_velocity')
    
    # real un/spliced
    adata.layers['unspliced']=adata.layers['real_unspliced']
    adata.layers['spliced']=adata.layers['real_spliced']
    scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    scv.tl.velocity(adata, vkey='real_velocity')
    scv.tl.velocity_graph(adata, vkey='real_velocity')
    scv.tl.velocity_embedding(adata, basis='umap', vkey='real_velocity')
    return adata

# Add the result from progeny (R, externally called)
def add_progeny(adata, donor):
    tab=pd.read_csv(data_path+'progeny/'+donor+'_progeny.csv', index_col=1)
    tab=tab.drop(tab.columns[0], axis=1)
    Z=[]
    for p in pd.unique(tab.Pathway):
        Z.append(tab[tab.Pathway==p].Activity.values)
    Z=np.array(Z)
    progeny_act = pd.DataFrame(Z.T, tab[tab.Pathway==p].index, columns=[x+'_progeny' for x in pd.unique(tab.Pathway)])
    adata.obs=pd.concat([adata.obs, progeny_act], axis=1)
    return adata

In [ ]:
# options
ccreg = True  # whether or not to regress out cell cycle (default: True)

# preparations
from pathlib import Path
Path(data_path+'by_donors/processed/').mkdir(parents=True, exist_ok=True)
ccr_suffix = '_ccreg' if ccreg else ''

# Process all samples
for donor in tqdm_notebook(donors):
    scv.settings.verbosity = 0
    adata = sc.read(data_path+'by_donors/SLAMv2_'+donor+'.h5')
    adata = prepare(adata)
    adata = annotate(adata, regress_cc=ccreg)
    adata = add_progeny(adata, donor)
    adata = embedd(adata)
    adata = velocity(adata)
    adata.obs.rename(columns={"Stem/TA 1": "Stem_TA 1", "Stem/TA 2": "Stem_TA 2", "Stem/TA 3": "Stem_TA 3"}, inplace=True)  # "/" throws error in saved .h5, so we rename these to "_"
    adata.write(data_path+'/by_donors/processed/SLAMv2_'+donor+'_processed'+ccr_suffix+'.h5')